In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [2]:
%load_ext autoreload
%autoreload 2
from pyha import *
from pyha.cores import *
from pyha.cores.util import *
import scipy
from scipy import signal
import warnings
warnings.filterwarnings('ignore') # yolo!
import numpy as np
from pyha.simulation.simulation import Simulator
# GitHub cannot display the Table of Contents!

ImportError: cannot import name 'QuartusHelper' from 'pyha.simulation.vhdl_simulation' (/home/gaspar/git/pyha/pyha/simulation/vhdl_simulation.py)

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Info
Works on 18-bit signed fixed-point numbers. Inputs and outputs assumed to be in range of -1 to 1. Outliers are saturated.

# Examples

## Noise removal

In [ ]:
np.random.seed(0)
N = 1024
t = np.linspace(0,2*2*np.pi,N)
signal = 0.8*np.sin(t)
noise = 0.1*np.random.normal(size=N)
input_signal = signal + noise # noisy input signal

dut = MovingAverage(window_len=128) # design we are testing
sim = Simulator(dut, trace=True).run(input_signal) # run simulations and gather trace
sim.plot_trace()

## Frequency response

In [ ]:
input_signal = [0.0] * 1024* 4
input_signal[0] = 1.0

dut = MovingAverage(window_len=128)
sim = Simulator(dut, trace=True).run(input_signal)
sim.plot_trace(mode={'Input': 'time', 'Output': 'frequency_response'})

## Matched filter

In [ ]:
samples_per_bit = 128
bits = [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0]
oversample = np.array([[x] * samples_per_bit for x in bits]).flatten()
input_signal = (oversample + np.random.normal(size=len(oversample))*0.5) * 0.25

dut = MovingAverage(window_len=samples_per_bit)
sim = Simulator(dut, trace=True).run(input_signal)
sim.plot_trace()

# RTL and NETLIST simulations

In [ ]:
# Pyha design can be converted to VHDL after the PYHA simulation has been ran
# VHDL sources can be used to run the RTL simulation, in addition NETLIST simulation is supported (after Quartus mapper)
# Pyha automatically manages everything needed if you specify the 'RTL', 'NETLIST' flags.
sim = Simulator(dut, output_dir=output_dir, extra_simulations=['RTL', 'NETLIST'])
sim.run(input_signal)

In [ ]:
output_dir = '~/Documents/pyha_output'
# conversion requires a simulation with 'train_model_for_conversion' flag enabled, which makes simulation much slower
input_signal = np.random.normal(size=8000) * 0.25
dut = MovingAverage(window_len=128)
sim = Simulator(dut, train_model_for_conversion=True).run(input_signal)

## RTL simulation

In [ ]:
from pyha.conversion.conversion import Conversion

In [ ]:
# convert sources and run the RTL simulation with GHDL and Cocotb
conversion = Conversion(dut, output_dir).to_vhdl()
sim = sim.run_rtl(conversion=conversion)

## Netlist simulation

## Make quartus project and netlist file

In [ ]:
# synthesise VHDL sources into a netlist file, this also yields initial resource usage
# netlist can also be simulated with GHDL/Cocotb, but it is super slow and quite useless (i have never seen it fail after sucessful synthesis)
from pyha.synthesis.quartus import make_quartus_project, QuartusDockerWrapper
make_quartus_project(conversion)
quartus = QuartusDockerWrapper(output_dir)
quartus.map()
quartus.eda()
# sim.set_netlist_file()
# sim.run_netlist(input_signal)

In [ ]:
sim.run_rtl(conversion=conversion, netlist=quartus.get_netlist_path())

## Test simulation equality

In [ ]:
# hardware simulations (PYHA, RTL, GATE) must match EXACTLY to pass.
# match to MODEL can be controlled with 'rtol' and 'atol'
sim.assert_simulations_equal(sims, rtol=1e-5, atol=1e-5)

## Resource usage and Fmax

In [ ]:
quartus.fit()
print(quartus.get_resource_usage('fit'))
print(quartus.get_fmax())